# PA7.1: Cross-Validation and Tuning

From Chapter 12 Course Book

In [12]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [13]:
ames = pd.read_csv("https://raw.githubusercontent.com/kevindavisross/data301/main/data/AmesHousing.txt", sep="\t")
ames

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,...,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,2926,923275080,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,1Fam,SLvl,6,6,1984,1984,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,TA,TA,CBlock,TA,TA,Av,GLQ,819.0,Unf,0.0,184.0,1003.0,...,Y,SBrkr,1003,0,0,1003,1.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,Detchd,1984.0,Unf,2.0,588.0,TA,TA,Y,120,0,0,0,0,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500
2926,2927,923276100,20,RL,NaN,8885,Pave,NaN,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,5,5,1983,1983,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,TA,TA,CBlock,Gd,TA,Av,BLQ,301.0,ALQ,324.0,239.0,864.0,...,Y,SBrkr,902,0,0,902,1.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1983.0,Unf,2.0,484.0,TA,TA,Y,164,0,0,0,0,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000
2927,2928,923400125,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,NaN,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,...,Y,SBrkr,970,0,0,970,0.0,1.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,80,32,

## Exercise 13.2.5: Four Models - Test Set Comparison

Consider four possible models for predicting house prices:

1. Using only the size and number of rooms.
2. Using size, number of rooms, and building type.
3. Using size and building type, and their interaction.
4. Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all four models.

In [14]:
# Single train/test split for all models
X = ames.drop("SalePrice", axis=1)
y = ames["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
# Model 1: Size and rooms only
ct1 = ColumnTransformer([
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
], remainder="drop")

pipeline1 = Pipeline([
    ("preprocessing", ct1),
    ("linear_regression", LinearRegression())
])

pipeline1.fit(X_train, y_train)
pred1 = pipeline1.predict(X_test)
mse1 = mean_squared_error(y_test, pred1)
rmse1 = np.sqrt(mse1)
print(f"Model 1 MSE: {mse1:.2f}")
print(f"Model 1 RMSE: {rmse1:.2f}")

Model 1 MSE: 3327468163.64
Model 1 RMSE: 57684.21


In [16]:
# Model 2: Size, rooms, and building type
ct2 = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
], remainder="drop")

pipeline2 = Pipeline([
    ("preprocessing", ct2),
    ("linear_regression", LinearRegression())
])

pipeline2.fit(X_train, y_train)
pred2 = pipeline2.predict(X_test)
mse2 = mean_squared_error(y_test, pred2)
rmse2 = np.sqrt(mse2)
print(f"Model 2 MSE: {mse2:.2f}")
print(f"Model 2 RMSE: {rmse2:.2f}")

Model 2 MSE: 3100121179.51
Model 2 RMSE: 55678.73


In [17]:
# Model 3: Size and building type with interaction
X_train3 = X_train[["Gr Liv Area", "Bldg Type"]]
X_test3 = X_test[["Gr Liv Area", "Bldg Type"]]

ct3 = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False, drop="first"), ["Bldg Type"]),
    ("passthrough", "passthrough", ["Gr Liv Area"])
])

pipeline3 = Pipeline([
    ("preprocessing", ct3),
    ("polynomial", PolynomialFeatures(degree=2, interaction_only=True)),
    ("linear_regression", LinearRegression())
])

pipeline3.fit(X_train3, y_train)
pred3 = pipeline3.predict(X_test3)
mse3 = mean_squared_error(y_test, pred3)
rmse3 = np.sqrt(mse3)
print(f"Model 3 MSE: {mse3:.2f}")
print(f"Model 3 RMSE: {rmse3:.2f}")

Model 3 MSE: 3012851227.74
Model 3 RMSE: 54889.45


In [18]:
# Model 4: 5-degree polynomial on size and rooms, plus building type
ct4 = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ("size", PolynomialFeatures(degree=5), ["Gr Liv Area"]),
    ("rooms", PolynomialFeatures(degree=5), ["TotRms AbvGrd"])
], remainder="drop")

pipeline4 = Pipeline([
    ("preprocessing", ct4),
    ("linear_regression", LinearRegression())
])

pipeline4.fit(X_train, y_train)
pred4 = pipeline4.predict(X_test)
mse4 = mean_squared_error(y_test, pred4)
rmse4 = np.sqrt(mse4)
print(f"Model 4 MSE: {mse4:.2f}")
print(f"Model 4 RMSE: {rmse4:.2f}")

Model 4 MSE: 3199162301.03
Model 4 RMSE: 56561.14


In [19]:
# Compare all models
results = pd.DataFrame({
    "Model": ["Model 1: Size + Rooms", 
              "Model 2: Size + Rooms + BldgType", 
              "Model 3: Size x BldgType interaction",
              "Model 4: Poly(5) Size + Poly(5) Rooms + BldgType"],
    "MSE": [mse1, mse2, mse3, mse4],
    "RMSE": [rmse1, rmse2, rmse3, rmse4]
})
results.sort_values("MSE")

,Model,MSE,RMSE
2,Model 3: Size x BldgType interaction,3.012851e+09,54889.445504
1,Model 2: Size + Rooms + BldgType,3.100121e+09,55678.731842
3,Model 4: Poly(5) Size + Poly(5) Rooms + BldgType,3.199162e+09,56561.137728
0,Model 1: Size + Rooms,3.327468e+09,57684.210696


**Answer:** Model 3 the lowest MSE and RMSE, indicating that it performed best.

## Exercise 13.3.1: Cross-Validation Comparison

Use the `cross_val_score()` function with the pipelines from Exercise 13.2.5 to find the cross-validated RMSE for each model. Which model do you prefer, and does it agree with your conclusion from earlier?

In [24]:
# Perform 5-fold cross-validation for each model

# Model 1: Size and rooms only
cv_scores1 = cross_val_score(pipeline1, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse1 = np.sqrt(-cv_scores1.mean())

# Model 2: Size, rooms, and building type
cv_scores2 = cross_val_score(pipeline2, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse2 = np.sqrt(-cv_scores2.mean())

# Model 3: Size and building type with interaction
# Note: Model 3 uses only specific columns
X_model3 = X[["Gr Liv Area", "Bldg Type"]]
cv_scores3 = cross_val_score(pipeline3, X_model3, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse3 = np.sqrt(-cv_scores3.mean())

# Model 4: 5-degree polynomial on size and rooms, plus building type
cv_scores4 = cross_val_score(pipeline4, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse4 = np.sqrt(-cv_scores4.mean())

# Compare cross-validation results
cv_results = pd.DataFrame({
    "Model": ["Model 1: Size + Rooms", 
              "Model 2: Size + Rooms + BldgType", 
              "Model 3: Size x BldgType interaction",
              "Model 4: Poly(5) Size + Poly(5) Rooms + BldgType"],
    "CV RMSE": [cv_rmse1, cv_rmse2, cv_rmse3, cv_rmse4],
    "Test RMSE": [rmse1, rmse2, rmse3, rmse4]
})

print("Cross-Validation Results:")
print(cv_results.sort_values("CV RMSE"))


Cross-Validation Results:
                                              Model       CV RMSE     Test RMSE
2              Model 3: Size x BldgType interaction  53583.840600  54889.445504
1                  Model 2: Size + Rooms + BldgType  54332.255227  55678.731842
0                             Model 1: Size + Rooms  56001.240238  57684.210696
3  Model 4: Poly(5) Size + Poly(5) Rooms + BldgType  56368.687251  56561.137728


**Answer:**

I still prefer model 3, this DOES agree with my prior conclusion as well

## Exercise 13.3.2: How Many Model Fitting Steps?

Recall that the model fitting step is when coefficients are computed for the linear regression.

How many different model fitting steps occurred when `gscv.fit(X, y)` was run?


**Answer:**

- Number of different degree values: 9 (degrees 1 through 9)
- Number of folds: 5
- Total model fits = 9 × 5 = **45 different model fitting steps**

For each of the 9 polynomial degrees, the model is fit 5 times (once for each cross-validation fold).

## Exercise 13.3.3: Grid Search with 100 Model Options

Consider one hundred modeling options for house price:

* House size, trying degrees 1 through 10
* Number of rooms, trying degrees 1 through 10
* Building Type

Hint: The dictionary of possible values that you make to give to GridSearchCV will have two elements instead of one.

Q1: Which model performed the best?

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

In [20]:
# Set up the column transformer and pipeline
ct = ColumnTransformer([
    ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
    ("size", PolynomialFeatures(), ["Gr Liv Area"]),
    ("rooms", PolynomialFeatures(), ["TotRms AbvGrd"])
], remainder="drop")

pipeline = Pipeline([
    ("preprocessing", ct),
    ("linear_regression", LinearRegression())
])

# Define parameter grid with TWO parameters
param_grid = {
    "preprocessing__size__degree": np.arange(1, 11),
    "preprocessing__rooms__degree": np.arange(1, 11)
}

# Use negative MSE as scoring (GridSearchCV maximizes, so we negate)
gscv = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_mean_squared_error")
gscv.fit(X, y)

,estimator,Pipeline(step...egression())])
,param_grid,"{'preprocessing__rooms__degree': array([ 1, 2..., 8, 9, 10]), 'preprocessing__size__degree': array([ 1, 2..., 8, 9, 10])}"
,scoring,'neg_mean_squared_error'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('dummify', ...), ('size', ...), ...]"


In [21]:
# Find best parameters
print("Best parameters:", gscv.best_params_)
print(f"Best MSE: {-gscv.best_score_:.2f}")
print(f"Best RMSE: {np.sqrt(-gscv.best_score_):.2f}")

Best parameters: {'preprocessing__rooms__degree': np.int64(1), 'preprocessing__size__degree': np.int64(3)}
Best MSE: 2798020988.10
Best RMSE: 52896.32


In [22]:
# Show top 10 models
results_df = pd.DataFrame(gscv.cv_results_)
top_models = results_df[["param_preprocessing__size__degree", 
                         "param_preprocessing__rooms__degree", 
                         "mean_test_score"]].copy()
top_models["MSE"] = -top_models["mean_test_score"]
top_models["RMSE"] = np.sqrt(top_models["MSE"])
top_models.sort_values("MSE").head(10)

,param_preprocessing__size__degree,param_preprocessing__rooms__degree,mean_test_score,MSE,RMSE
2,3,1,-2.798021e+09,2.798021e+09,52896.323011
12,3,2,-2.805308e+09,2.805308e+09,52965.154426
22,3,3,-2.822697e+09,2.822697e+09,53129.064117
32,3,4,-2.840668e+09,2.840668e+09,53297.913826
62,3,7,-2.841414e+09,2.841414e+09,53304.915999
42,3,5,-2.864656e+09,2.864656e+09,53522.482566
52,3,6,-2.876530e+09,2.876530e+09,53633.295407
30,1,4,-2.891811e+09,2.891811e+09,53775.564021
40,1,5,-2.894181e+09,2.894181e+09,53797.591423
1,2,1,-2.921754e+09,2.921754e+09,54053.252747


**Q1 Answer:** The best model with lowest RMSE uses the degree 3 for size and degree 1 for rooms, shown above.

**Q2 Answer: Downsides of trying all possible model options:**

1. With 10 × 10 = 100 combinations and 5 folds, we fit 500 models total. This takes significant time and resources.
Also, testing so many combinations increases the chance of finding a model that performs well on our specific dataset by chance rather than true predictive power. Furthermore, higher degree polynomials may not provide much improvement and can lead to overfitting.

To go about choosing a smaller number of values to try:

I would test each feature separately first, so I would find the best degree for size alone, then for rooms alone, then combine them.